In [1]:
import sys
import os

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1595797161173_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas")
sc.install_pypi_package("plotly")
sc.install_pypi_package("matplotlib")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


  Running setup.py bdist_wheel for retrying: started
  Running setup.py bdist_wheel for retrying: finished with status 'done'
  Stored in directory: /var/lib/livy/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built retrying


In [3]:
MAX_MEMORY = "14g"
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ECom').config("spark.driver.memory", MAX_MEMORY).getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import to_timestamp, hour, dayofweek.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# loading the dataset from S3 - 2019-Oct.csv
df = spark.read.csv("s3://mlc-c5-assignment/2019-Oct.csv", header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# exploring the dataframe - top 20 rows
df.show(20, False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|event_time             |event_type|product_id|category_id        |category_code                      |brand   |price  |user_id  |user_session                        |
+-----------------------+----------+----------+-------------------+-----------------------------------+--------+-------+---------+------------------------------------+
|2019-10-01 00:00:00 UTC|view      |44600062  |2103807459595387724|null                               |shiseido|35.79  |541312140|72d76fde-8bb3-4e00-8c23-a032dfed738c|
|2019-10-01 00:00:00 UTC|view      |3900821   |2053013552326770905|appliances.environment.water_heater|aqua    |33.2   |554748717|9333dfbd-b87a-4708-9857-6336556b0fcc|
|2019-10-01 00:00:01 UTC|view      |17200506  |2053013559792632471|furniture.living_room.sofa         |null    |543.1  |519107250|566511c2-e2e3-422b-b695-cf8e6e

In [7]:
# exploring the dataframe - schema
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- event_time: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

In [8]:
# Number of rows and columns in the dataset
print((df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(42448764, 9)

In [9]:
# 5 most popular products sold
df[df["event_type"]=="purchase"].groupby('product_id').count().sort(col("count").desc()).show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+
|product_id|count|
+----------+-----+
|   1004856|28944|
|   1004767|21806|
|   1004833|12697|
|   1005115|12543|
|   4804056|12381|
+----------+-----+
only showing top 5 rows

In [10]:
# 5 most popular brands
df[df["event_type"]=="purchase"].groupby('brand').count().sort(col("count").desc()).show(6) #showing 6 as null is also there

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|  brand| count|
+-------+------+
|samsung|172896|
|  apple|142873|
|   null| 58214|
| xiaomi| 56616|
| huawei| 23501|
|lucente| 11578|
+-------+------+
only showing top 6 rows

In [11]:
# Number of unique users
df.select(countDistinct("user_id")).withColumnRenamed("count(DISTINCT user_id)", "UniqueUsers").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+
|UniqueUsers|
+-----------+
|    3022290|
+-----------+

In [12]:
# The most active user on the platform
df.groupby('user_id').agg(countDistinct("user_session")).withColumnRenamed("count(DISTINCT user_session)", "UniqueSessions").sort(col("UniqueSessions").desc()).show(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------+
|  user_id|UniqueSessions|
+---------+--------------+
|512475445|          7400|
+---------+--------------+
only showing top 1 row

In [13]:
# Average and Maximum price for smartphones purchased by the customers
df.filter((col("event_type")=="purchase") & (col("category_code").like("electronics.smartphone"))).groupBy().max('price').show()
df.filter((col("event_type")=="purchase") & (col("category_code").like("electronics.smartphone"))).groupBy().avg('price').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+
|max(price)|
+----------+
|   2110.45|
+----------+

+-----------------+
|       avg(price)|
+-----------------+
|464.6191130945662|
+-----------------+

In [14]:
# Event-type funnel distribution
# Remember Spark dataframe cannot be visualised directly
events_count = df.groupby('event_type').count().withColumnRenamed("count", "event_count")
labels = [row.event_type for row in events_count.collect()]
size = [int(row.event_count) for row in events_count.collect()]
print(labels, size)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['cart', 'view', 'purchase'] [926516, 40779399, 742849]

In [44]:
# from plotly import graph_objects as go
# fig = go.Figure(go.Funnel(y=size, x=labels))
# fig.show()
from plotly import express as px
fig = px.funnel_area(names=labels,
                    values=size)
fig.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df = df.withColumn("event_time", to_timestamp(df.event_time, 'yyyy-MM-dd HH:mm:ss'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Traffic on different days of the week
# Remember Spark dataframe cannot be visualised directly
day_visits = df.groupby(dayofweek(df["event_time"])).agg(countDistinct("user_session"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
day_visits =  day_visits.withColumnRenamed("dayofweek(event_time)","DayOfWeek").withColumnRenamed("count(DISTINCT user_session)","ActivityCount")
day = [row.DayOfWeek for row in day_visits.collect()]
activity = [int(row.ActivityCount) for row in day_visits.collect()]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
day_visits.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------------+
|DayOfWeek|ActivityCount|
+---------+-------------+
|        1|      1236041|
|        6|      1286835|
|        3|      1504737|
|        4|      1476124|
|        5|      1413669|
|        2|      1173112|
|        7|      1191891|
+---------+-------------+

In [22]:
import matplotlib.pyplot as plt
plt.clf()
day_visits_pd = day_visits.toPandas().sort_values(by=['DayOfWeek'], inplace = True)
day_visits_pd.plot(kind='bar', x='DayOfWeek', y='ActivityCount')
plt.xticks(day_visits_pd.DayOfWeek)
plt.xlim(0, 7)
plt.xlabel('Day')
plt.ylabel('Number of visits')

%matplot plt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'plot'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'plot'



In [ ]:
# Handling missing values 

##Check Missing values for each column
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns if c!='event_time']).show()

In [ ]:
##Check Missing values for each column when the event was Purchase
df.filter(col("event_type")=="purchase").select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns if c!='event_time']).show()

####Since the values having null does not change the distribution and after dropping them we still have enough data to build the model. Hence dropping

In [24]:
df = df.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Checking for duplicate entries
if df.count() > df.dropDuplicates().count():
    print('Data has duplicates')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Data has duplicates

In [26]:
df = df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
columns_to_drop = ['category_id']
df = df.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Generating 2 columns from category code
# Columns have only single entity (electronics.video.tv: electronics, video)
# Check if split command works with '.' or Spark treats it as special character
from pyspark.sql.functions import split
df = df.withColumn("Category", split(col("category_code"), "\.").getItem(0)) \
 .withColumn("SubCategory", split(col("category_code"), "\.").getItem(1)).drop("category_code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Check if the column is added
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['event_time', 'event_type', 'product_id', 'brand', 'price', 'user_id', 'user_session', 'Category', 'SubCategory']

In [30]:
# Activities in a session by the user
# Window functions can be helpful here
from pyspark.sql.window import Window
w = Window.partitionBy(df.user_session)
df = df.withColumn("ActivityCount", count('event_type').over(w))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
# View count for a product by the user
# Window functions can be helpful here
w = Window.partitionBy(df.user_id, df.product_id)
df = df.withColumn("ProductView", count(when(col('event_type')=='view', col('event_type'))).over(w))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
# View count for the secondary category by the user
# Window functions can be helpful here
w = Window.partitionBy(df.user_id, df.SubCategory)
df = df.withColumn('SubCategoryViewCount', count(when(col('event_type')=='view', col('event_type'))).over(w))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# Average shopping expense for a product category
# Window functions can be helpful here
from pyspark.sql.functions import avg
w = Window.partitionBy(df.SubCategory)
df = df.withColumn("AvgSpendingOnCategoty", avg(when(col("event_type")=="purchase", df.price)).over(w))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Session count for a user
# Window functions can be helpful here
df.groupby('user_id').agg(countDistinct("user_session")).withColumnRenamed("count(DISTINCT user_session)", "NumberOfSessions").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+
|  user_id|NumberOfSessions|
+---------+----------------+
|513973242|               2|
|512496671|               3|
|515124387|               6|
|540228262|              13|
|531517576|               4|
|515604115|              22|
|512598373|               5|
|522539566|              16|
|540000378|              13|
|518619616|               9|
|557429591|               7|
|529079049|              13|
|564004534|               2|
|514928063|               5|
|557617569|              48|
|529835172|               9|
|562615028|               2|
|517205156|               8|
|556825423|              22|
|513924758|               4|
+---------+----------------+
only showing top 20 rows

In [35]:
# Generating the hour variable
df = df.withColumn("EventHour", hour(col('event_time')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Reduction in brands for analysis: Top 20 + ‘others'
top_twenty = df[df["event_type"]=="purchase"].groupby('brand').count().sort(col("count").desc()).limit(20).select('brand').collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
top_twenty = [row.brand for row in top_twenty]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
df = df.withColumn("brand", when(col('brand').isin(top_twenty), col('brand')).otherwise('others'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
# Generating 'is_purchased' variable
df = df.filter(df['event_type'] !='view')
df = df.withColumn("is_purchased", when(df['event_type']=='purchase', 1).otherwise(0))

from pyspark.sql.functions import max as sparkMax
w = Window.partitionBy(df.user_session, df.product_id)
df = df.withColumn("is_purchased", sparkMax(col('is_purchased')).over(w))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Dropping redundant rows
df = df.filter(df['event_type'] =='cart')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Printing the top 20 rows as output
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+----------+-------+-------+---------+--------------------+-----------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+
|         event_time|event_type|product_id|  brand|  price|  user_id|        user_session|   Category|SubCategory|ActivityCount|ProductView|SubCategoryViewCount|AvgSpendingOnCategoty|EventHour|is_purchased|
+-------------------+----------+----------+-------+-------+---------+--------------------+-----------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+
|2019-10-16 12:10:16|      cart|   1004870|samsung| 287.65|552386415|0000ea0f-375d-428...|electronics| smartphone|            4|          3|                   4|   464.33546803889675|       12|           1|
|2019-10-15 15:04:34|      cart|   1004856|samsung| 131.66|555270321|0000f9ba-8fb7-49d...|electronics| smartphone|            2|          1|                  21|   464.3354

In [42]:
# Dropping the redundant columns 
redundant_cols = ["event_time","event_type", "user_id", "user_session","product_id"]
df = df.drop(*redundant_cols)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# Check if only the required columns are present to build the model
# If not, drop the redundant columns
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['brand', 'price', 'Category', 'SubCategory', 'ActivityCount', 'ProductView', 'SubCategoryViewCount', 'AvgSpendingOnCategoty', 'EventHour', 'is_purchased']

In [42]:
# Storing the transformed df in S3 bucket to prevent repetition of steps again
df.write.csv('s3://asr-aiml-2020/processed.csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
df1 = spark.read.csv("s3://asr-aiml-2020/processed.csv", header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+------------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+
|  brand|  price|    Category|SubCategory|ActivityCount|ProductView|SubCategoryViewCount|AvgSpendingOnCategoty|EventHour|is_purchased|
+-------+-------+------------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+
|samsung| 172.14| electronics| smartphone|            5|         10|                  76|   464.33546803888913|       22|           1|
|samsung| 172.14| electronics| smartphone|            5|         10|                  76|   464.33546803888913|       22|           1|
| others|  55.83|construction|      tools|            2|          1|                   1|   130.60873519227255|        9|           0|
|samsung| 131.51| electronics| smartphone|            3|          5|                  16|   464.33546803888913|        5|           0|
| others| 190.45| electronics|      video|           15

In [38]:
# Categorising the attributes into its type - Continuous and Categorical
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import IntegerType
bucketizer = Bucketizer(splits=[ 0, 6, 12, 18 , 24, float('Inf') ], inputCol="EventHour", outputCol="EventHour_Buckets")
df1 = bucketizer.setHandleInvalid("keep").transform(df1)
df1 = df1.withColumn("EventHour_Buckets", df1["EventHour_Buckets"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
# Feature transformation for categorical features
brand_indexr = StringIndexer(inputCol='brand',   outputCol='brand_idx')   
category_indexr = StringIndexer(inputCol='Category',   outputCol='Category_idx') 
subCat_indexr = StringIndexer(inputCol='SubCategory',   outputCol='SubCategory_idx') 
one_hot_encoder = OneHotEncoderEstimator(inputCols=['brand_idx', 'Category_idx', 'SubCategory_idx'],
                             outputCols=['brand_Enc', 'Category_enc', 'SubCategory_enc'])

# Vector assembler to combine all the features
assembler = VectorAssembler(inputCols=['brand_Enc' ,'Category_enc', 'SubCategory_enc', 'price',"ActivityCount", "ProductView", "SubCategoryViewCount", "EventHour_Buckets", "AvgSpendingOnCategoty"],
                            outputCol="features")
# Pipeline for the tasks
pipeline = Pipeline(stages=[brand_indexr,category_indexr,subCat_indexr, one_hot_encoder, assembler])

# Transforming the dataframe df
df1 = pipeline.fit(df1).transform(df1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# Checking the elements of the transformed df - Top 20 rows
df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+------------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+-----------------+---------+------------+---------------+--------------+--------------+---------------+--------------------+
|  brand|  price|    Category|SubCategory|ActivityCount|ProductView|SubCategoryViewCount|AvgSpendingOnCategoty|EventHour|is_purchased|EventHour_Buckets|brand_idx|Category_idx|SubCategory_idx|     brand_Enc|  Category_enc|SubCategory_enc|            features|
+-------+-------+------------+-----------+-------------+-----------+--------------------+---------------------+---------+------------+-----------------+---------+------------+---------------+--------------+--------------+---------------+--------------------+
|samsung| 172.14| electronics| smartphone|            5|         10|                  76|   464.33546803887987|       22|           1|                3|      0.0|         0.0|            0.0|(20,[0],[1.0])|(12,[0],[1.0])| (

In [46]:
df1.write.parquet("s3://asr-aiml-2020/processed.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…